# Querying SQL databases

Your task is to analyze game and player statistics from the NBA 2008-2009 season provided as a relational database of play-by-play data.

You cannot use jupyter to execute the queries. To write and test the queries, use the web interface at http://pollux.vo.elte.hu/basketball/. Copy queries into the worksheet for submission.

## Excercise 1 - Simple queries

a) List all teams playing in the league during the season

b) List teams in alphabetic order but division by division

c) Count the number of persons/players in the league

In [4]:
import pymysql

In [7]:
db = pymysql.connect("http://pollux.vo.elte.hu/")

OperationalError: (2003, "Can't connect to MySQL server on 'http://pollux.vo.elte.hu/' ([Errno -2] Name or service not known)")

In [3]:
myDB = MySQLdb.connect(host="http://pollux.vo.elte.hu/basketball/",port=80)
cHandler = myDB.cursor()
cHandler.execute("SHOW DATABASES")
results = cHandler.fetchall()
for items in results:
    print(items[0])

 

ModuleNotFoundError: No module named 'MySQLdb'

In [ ]:
c = conn.cursor()
c.execute('''
          Select * from dbo.shots
          ''')

In [ ]:
SELECT name FROM dbo.team

In [ ]:
select * from dbo.team order by division_id, name

In [ ]:
select count(person_id) from dbo.player

## Excercise 2 - Simple joins

a) List teams by division, including the division name

b) List players by team including the uniform number

In [ ]:
select division_id,name from dbo.team order by division_id

In [ ]:
SELECT team.name, player.number, person.name
FROM team
INNER JOIN player ON player.team_id = team.id
INNER JOIN person ON person.id = player.person_id
ORDER BY 1, 2

## Excercise 3 - Simple aggregation

a) Count number of persons playing for a given team

b) List all teams along with the number of players

c) Count the number of teams a given person played for

d) List all players along with the number of teams they played for

e) List players who played for more than two teams during the season

In [ ]:
select count(person_id) from dbo.player where team_id='ATL'

In [ ]:
select count(person_id) from dbo.player group by team_id

In [ ]:
select count(team_id) from player where person_id=4

In [ ]:
SELECT person.name, COUNT(*)
FROM person
INNER JOIN player ON player.person_id = person.id
INNER JOIN team ON team.id = player.team_id
GROUP BY person.name

In [ ]:
SELECT person.name, COUNT(*)
FROM person
INNER JOIN player ON player.person_id = person.id
GROUP BY person.name
HAVING COUNT(*) > 2

## Excercise 4 - More complex queries with set operations and common table expressions

a) Pick a game and list all field shots and free throw attempts, including points

b) Combine the previous two into a single table and only show successful attempts

c) Take previous query and sum all points by team so that end result is available

d) Remove filter on game and get end result of each game during season

e) Modify the previous query to return a single line per game, with home team and away team points

In [8]:
SELECT count(*) from freethrow
INNER JOIN game ON game.id = freethrow.game_id
where game_id=10
SELECT count(*) from shot
INNER JOIN game ON game.id = shot.game_id
where game_id=10

SyntaxError: invalid syntax (<ipython-input-8-e4209fa218f6>, line 1)

In [ ]:
select count(*) from (SELECT * from point
where game_id=10 ) as a
inner join (SELECT * from freethrow where game_id=10 ) as b on b.cntr = a.cntr


In [ ]:
select * from shot as s
left outer join point as p
  on p.game_id = s.game_id AND p.cntr = s.cntr
where s.game_id = 10

In [ ]:
select * from shot as s
where s.game_id = 10
and cntr in (select cntr from point where game_id =10)

In [ ]:
select * from 
(
( select game_id, cntr, team_id, player_id, points from shot as s
where s.game_id = 10
and cntr in (select cntr from point where game_id =10) )

union all

(select game_id, cntr,  team_id, player_id, points from freethrow as f
where f.game_id = 10
and cntr in (select cntr from point where game_id =10))
) as a

In [ ]:
select team_id, sum(points) from 

(
(
select team_id, player_id, points from 
(
select  * from shot as s
where s.game_id = 10
and cntr in (select cntr from point where game_id =10) 
) as sh
inner join point on point.game_id = sh.game_id and point.cntr = sh.cntr
) 

union all

(
select team_id, player_id, points from 
(
select  * from freethrow as f
where f.game_id = 10
and cntr in (select cntr from point where game_id =10) 
) as fh
inner join point on point.game_id = fh.game_id and point.cntr = fh.cntr
) 
) as uu 

group by uu.team_id

In [ ]:
with uu as
(
(
select sh.game_id, sh.team_id, sh.player_id, points from 
(
select  * from shot as s
where cntr in (select cntr from point) 
) as sh
inner join point on point.game_id = sh.game_id and point.cntr = sh.cntr
) 

union all

(
select fh.game_id, fh.team_id, fh.player_id, points from 
(
select  * from freethrow as f
where cntr in (select cntr from point) 
) as fh
inner join point on point.game_id = fh.game_id and point.cntr = fh.cntr
) 
)
select game_id, team_id, sum(points) from uu
group by game_id, team_id
order by 1 

# e feladat
with shh as (
select  s.game_id, team_id, points from shot as s
inner join point on point.game_id = s.game_id and point.cntr = s.cntr 
UNION ALL
select  f.game_id, team_id, points from freethrow as f
inner join point on point.game_id = f.game_id and point.cntr = f.cntr ) 
,
res as (select game_id, sum(points) as points, team_id  from shh
group by game_id, team_id)
, 
t as (
select r1.*, r2.team_id as team2, r2.points as points2  from res as r1
inner join res as r2 on r2.game_id = r1.game_id and r2.team_id <> r1.team_id  )

select date, t.points, t.team_id as home_team, t.team2 as away_team, t.points2 from game
inner join t on t.game_id = game.id and t.team_id = game.home_team_id
order by 2










## Excercise 5 - a rather complex example

a)  Print season results by division. For each team, calculate with a **single_ query**

- number of games played
- wins
- losts
- winning percentage

Compare with https://en.wikipedia.org/wiki/2008–09_NBA_season#By_division

In [ ]:
# query